In [55]:
import pandas as pd
import numpy as np
import zipfile
import os

from scipy.stats import entropy, skew
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report

In [56]:
zip_files = 'homework.zip'

with zipfile.ZipFile(zip_files, 'r') as zip_ref:
    zip_ref.extractall('unzip_folder')

In [50]:
def load_data_frame(folder_path):

    data = pd.DataFrame()
    type_activity = os.path.basename(folder_path)
    for filename in os.listdir(folder_path):

      if filename.endswith('.csv'):
        df = pd.read_csv(os.path.join(folder_path, filename))
        data_d = pd.DataFrame(
        {
        'max_x': df['accelerometer_X'].max(),
        'min_x': df['accelerometer_X'].min(),
        'mean_x': df['accelerometer_X'].mean(),
        'std_x': df['accelerometer_X'].std(),

        'max_y': df['accelerometer_Y'].max(),
        'min_y': df['accelerometer_Y'].min(),
        'mean_y': df['accelerometer_Y'].mean(),
        'std_y': df['accelerometer_Y'].std(),

        'max_z': df['accelerometer_Z'].max(),
        'min_z': df['accelerometer_Z'].min(),
        'mean_z': df['accelerometer_Z'].mean(),
        'std_z': df['accelerometer_Z'].std(),
        'type_activity': type_activity,
        }, index=[0]
        )
        data = pd.concat([data, data_d]).reset_index(drop=True)

    return data

In [15]:
walk_path = 'unzip_folder/data/walking'
stairs_path = 'unzip_folder/data/stairs'
run_path = 'unzip_folder/data/running'
idle_path = 'unzip_folder/data/idle'


walk_df = load_data_frame(walk_path)
stairs_df = load_data_frame(stairs_path)
run_df = load_data_frame(run_path)
idle_df = load_data_frame(idle_path)


In [51]:
df = pd.concat([walk_df, stairs_df, run_df, idle_df]).reset_index(drop=True)

In [52]:
df

,max_x,min_x,mean_x,std_x,max_y,min_y,mean_y,std_y,max_z,min_z,mean_z,std_z,type_activity
0,5.750873,-13.445837,-4.115952,4.422851,9.270349,-20.590134,-9.787816,5.767905,22.821531,-14.705187,-0.267991,6.114929,walking
1,4.721366,-7.244854,-0.544122,2.991292,-0.263362,-22.888569,-9.388942,5.793207,21.629220,-20.973207,0.279164,7.566246,walking
2,2.609680,-14.973337,-3.825296,4.137669,-1.537077,-22.471977,-10.441273,4.340473,5.310340,-13.302185,-1.897325,4.567108,walking
3,2.748544,-12.033257,-3.220042,3.734462,6.320693,-17.310078,-8.775547,4.978474,32.541990,-14.053964,1.196622,7.525876,walking
4,4.869807,-20.676327,-4.179478,5.764550,2.336741,-26.197355,-9.627245,6.636436,17.913418,-21.265299,-1.357512,7.565687,walking
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6457,0.464475,-1.043872,-0.093693,0.308527,5.726931,3.888184,4.884491,0.531570,9.097966,7.445967,8.344112,0.465664,idle
6458,0.301669,0.134075,0.244368,0.031550,0.028730,-0.124498,-0.026655,0.032736,9.797073,9.720459,9.761480,0.019058,idle
6459,0.383072,0.244209,0.278366,0.031731,-0.028730,-0.138864,-0.074539,0.022971,9.792285,9.720459,9.765310,0.018587,idle
6460,0.799663,-0.454898,0.360886,0.290793,7.101202,4.946421,6.557559,0.410700,8.504205,6.636728,7.118600,0.381396,idle


In [53]:
X = df.drop('type_activity', axis=1)
y = df['type_activity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [54]:
# Random Forest
model_rf = RandomForestClassifier(n_estimators=100)
model_rf.fit(X_train, y_train)
predictions_rf = model_rf.predict(X_test)
print("Random Forest:")
print(classification_report(y_test, predictions_rf))

# SVM
model_svm = svm.SVC()
model_svm.fit(X_train, y_train)
predictions_svm = model_svm.predict(X_test)
print("SVM:")
print(classification_report(y_test, predictions_svm))

Random Forest:
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00       190
     running       1.00      1.00      1.00       677
      stairs       1.00      1.00      1.00        35
     walking       1.00      1.00      1.00       391

    accuracy                           1.00      1293
   macro avg       1.00      1.00      1.00      1293
weighted avg       1.00      1.00      1.00      1293

SVM:
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00       190
     running       1.00      1.00      1.00       677
      stairs       1.00      0.54      0.70        35
     walking       0.96      1.00      0.98       391

    accuracy                           0.99      1293
   macro avg       0.99      0.89      0.92      1293
weighted avg       0.99      0.99      0.99      1293



Обидві моделі добре підходять для цього типу задачі. RandomForest показала себе краще, ніж SVM для цього набору даних. Вона демонструє відмінну точність, повноту і F1-score 1.0 для всіх класів.

In [46]:
# Random Forest
param_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

random_rf = RandomizedSearchCV(RandomForestClassifier(), param_rf, n_iter=10, cv=5)
random_rf.fit(X_train, y_train)

print("Best parameters for Random Forest: ", random_rf.best_params_)

# SVM
param_svm = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
}

random_svm = RandomizedSearchCV(svm.SVC(), param_svm, n_iter=10, cv=5)
random_svm.fit(X_train, y_train)

print("Best parameters for SVM: ", random_svm.best_params_)

Best parameters for Random Forest:  {'n_estimators': 200, 'min_samples_split': 2, 'max_depth': 20}
Best parameters for SVM:  {'kernel': 'poly', 'gamma': 1, 'C': 0.1}


RandomizedSearchCV працює дуже схоже до GridSearchCV, але замість того, щоб пробувати всі можливі комбінації параметрів, він випадковим чином вибирає певну кількість комбінацій. Це може бути набагато швидше, особливо якщо у вас багато параметрів для налаштування.